In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy

#importing prophet
from fbprophet import Prophet




import pandas as pd
import numpy as np
from fbprophet import Prophet
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import add_changepoints_to_plot

#Visualizing Interactive Plots
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go
from datetime import datetime

from datetime import timedelta, date

start_date = date(2017, 1, 3)
today_date = datetime.today().strftime('%Y-%m-%d')
end_date = date(2020, 1, 1)
dataframeall = pd.DataFrame()
dataframeall = pd.concat([pd.DataFrame({'Date': pd.date_range(start_date, end_date, freq='d')}, columns=['Date'])])
dataframeall = dataframeall[:-1]


# Initialize plotly
init_notebook_mode(connected=True)

%matplotlib inline

#weekend
def weekend(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 5 or date.weekday() == 6:
        return 1
    else:
        return 0

#usHolidays
usHolidays = pd.DataFrame({
  'holiday': 'usHolidays',
  'ds': pd.to_datetime(['2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04',
               '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24',
               '2016-12-26', '2017-01-02', '2017-01-16', '2017-02-20',
               '2017-05-29', '2017-07-04', '2017-09-04', '2017-10-09',
               '2017-11-10', '2017-11-23', '2017-12-25', '2018-01-01',
               '2018-01-15', '2018-02-19', '2018-05-28', '2018-07-04',
               '2018-09-03', '2018-10-08', '2018-11-12', '2018-11-22',
               '2018-12-25', '2019-01-01', '2019-01-21', '2019-02-18',
               '2019-05-27', '2019-07-04', '2019-09-02', '2019-10-14',
               '2019-11-11', '2019-11-28', '2019-12-25','2019-12-31']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays =usHolidays
holidays

#plotly_df
def plotly_df(df, title=''):
    """Visualize all the dataframe columns as line plots."""
    common_kw = dict(x=df.index, mode='lines')
    data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)


# Load dataset
cust_orders = pd.read_csv("5l_customer_orders_127.csv", index_col= 'Group')
#fill with 0
cust_orders_head=cust_orders.fillna(0)
#create dataframe
df = pd.DataFrame(cust_orders_head)

df['Date'] =pd.to_datetime(df.Date)
df = df.sort_values('Date')

today_date = datetime.today().strftime('%Y-%m-%d')
mask = (df['Date'] < today_date)
df_mask = df.loc[mask]


df_wo_wknd_re = df_mask
#print('df_wo_wknd_re ',df_wo_wknd_re)
df_wo_wknd_re = df_wo_wknd_re.reset_index()

df_wo_wknd_re = df_wo_wknd_re.set_index('Date')


# for testing data
#print(df_mask)
df_mask['weekend'] = df_mask['Date'].apply(weekend)
df_mask['weekend']
df_wo_wknd = df_mask[df_mask.weekend != 1]
df_wo_wknd_re_test = df_wo_wknd
df_wo_wknd_re_test = df_wo_wknd_re_test.reset_index()

df_wo_wknd_re_test = df_wo_wknd_re_test.set_index('Date')

group_code_unique = df_wo_wknd_re['Group'].unique()


start_date = today_date
end_date = date(2019, 12, 31)
test_df_all = pd.DataFrame()
test_df_all = pd.concat([pd.DataFrame({'Date': pd.date_range(start_date, end_date, freq='d')}, columns=['Date'])])
test_df_all

master_index=0
#for each master/group code, genrate loop
for master in group_code_unique:
    print(master)
    df_wo_wknd_ele = df_wo_wknd_re.loc[df_wo_wknd_re['Group'] == master]

    df_wo_wknd_ele = df_wo_wknd_ele.reset_index()

    df_wo_wknd_ele = df_wo_wknd_ele.drop(['Group'],axis=1)
    df_wo_wknd_ele_tr = df_wo_wknd_ele.transpose()
    
    grouped = df_wo_wknd_ele.groupby(['Date'])['Quantity'].apply(np.sum)
    
    grouped_df =pd.DataFrame(grouped,columns= [ 'Quantity'])
    grouped_df.columns =[master]
    
    
    #for test data
    
    df_wo_wknd_ele_test = df_wo_wknd_re_test.loc[df_wo_wknd_re_test['Group'] == master]

    df_wo_wknd_ele_test = df_wo_wknd_ele_test.reset_index()

    df_wo_wknd_ele_test = df_wo_wknd_ele_test.drop(['Group'],axis=1)
    #df_wo_wknd_ele_tr = df_wo_wknd_ele.transpose()
    
    grouped_test = df_wo_wknd_ele_test.groupby(['Date'])['Quantity'].apply(np.sum)
    
    grouped_df_test =pd.DataFrame(grouped_test,columns= [ 'Quantity'])
    grouped_df_test.columns =[master]
    
    # Splitting Train test data for individual Master codes
    train_size = int(len(grouped_df))
    #test_size = int(len(test_df_all))
    train = grouped_df[:train_size]
    #test = test_df_all
    train['Date'] = train.index
    train['Date'] = train['Date'].astype('datetime64[ns]')
    train = train.set_index('Date')
    #test['Date'] = test.index
    #test['Date'] = test['Date'].astype('datetime64[ns]')
    #test = test.set_index('Date')
    #test_transposed = test.transpose()
    #test['ds'] = test.index
    #test['ds'] = test['ds'].astype('datetime64[ns]')
    train_transposed = train.transpose()
    #test_transposed = test.transpose()
    # Prediction size
    prediction_size = int(len(test_df_all))
    new = pd.DataFrame(train_transposed.iloc[0])
    #ax = plt.gca()
    
    # create 'ds' and 'y' elements for Prophet ML algo
    new['ds'] = train_transposed.columns
    new['ds'] = new['ds'].astype('datetime64[ns]')
    new.columns = ['y', 'ds']
    new_table = new[['ds','y']]
    new_table.set_index('ds')
    
    #create Model by calling Prophet
    my_model = Prophet(interval_width=0.95, holidays=holidays)
    
    #Add Daily Seasonality
    my_model.add_seasonality(name='daily', period=1, fourier_order=4)
    
    #Plot Graph with existing data
    #plotly_df(new,'Daily Shipment Seasonality')
    
    # Fit model on train data
    my_model.fit(new_table)
    # Make Future Dataframe with daily frequency and Prediction size
    future_dates = my_model.make_future_dataframe(periods = prediction_size, freq ="d")
    temp_history_dates = future_dates[future_dates['ds']<today_date]
    temp_future_dates = future_dates[future_dates['ds']>=today_date]
    temp_future_dates['weekend'] = temp_future_dates['ds'].apply(weekend)
    temp_future_dates = temp_future_dates[temp_future_dates.weekend != 1]
    
    temp_future_dates = temp_future_dates.drop(['weekend'], axis=1)
    future_dates = pd.merge(temp_history_dates, temp_future_dates, on=['ds'] , how='outer')
    future_dates = future_dates.reset_index()
    
    forecast = my_model.predict(future_dates)
    forecast = round(forecast)
    
    #Plot Forecast
    #figure = my_model.plot(forecast)
    #for changepoint in my_model.changepoints:
    #    plt.axvline(changepoint,ls='--', lw=1)

    # Merge forecasts with given IDs for Model 1
    #test_merge = pd.merge(forecast, test, on=['ds'] , how='outer')
    #cols = ['ds','yhat']
    #test_new = test_merge[cols]
    cols = ['ds','yhat']
    test_new = forecast[cols]
    test_new.yhat[test_new.yhat < 0] = 0

    # Forecasted data
    fc = test_new[['ds', 'yhat']].copy()
    fc[master] = fc['yhat']
    fc =  fc.drop('yhat', axis=1)
    fc['ds'] = fc['ds'].astype('datetime64[ns]')

    fc_transposed = fc.transpose()
    fc_transposed.columns = fc.index.values
    
    print('length::: ',len(grouped_df ))
    
    test_val = fc[fc['ds'] >= today_date]
    #print('test_val ',test_val)
    #fc_re['weekend'] = fc_re['ds'+master].apply(weekend)
    #fc_re = fc_re[fc_re.weekend != 1]
    grouped_df = grouped_df.reset_index()
    grouped_df_transposed = grouped_df.transpose()
    if master_index == 0:
        idx = 0
        columns = train.columns
        fc_re = pd.DataFrame(columns=columns)
        grouped_df_transposed_re = pd.DataFrame(columns=columns)
        #new_col = fc_transposed.iloc[master_index] #be a list, a Series, an array or a scalar 
        new_col = pd.Series()
        new_col = pd.to_datetime(pd.Series(dataframeall['Date'])) #be a list, a Series, an array or a scalar 
        fc_re.insert(loc=idx, column='dates', value=new_col)
        fc_re['ds'+master] = fc_transposed.iloc[master_index].astype('datetime64[ns]')
        
        #grouped_df_transposed_re
        grouped_df_transposed_re.insert(loc=idx, column='dates', value=new_col)
        grouped_df_transposed_re['ds'+master] = grouped_df_transposed.iloc[master_index].astype('datetime64[ns]')
    else:
        fc_re['ds'+master] = fc['ds']
        grouped_df_transposed_re['ds'+master] = grouped_df['Date']
    
    #test_merge = pd.merge(forecast, test, on=['ds'] , how='outer')
    fc_re[master] = fc[master]
    grouped_df_transposed_re[master] = grouped_df[master]
    
    #print(fc_re[master])

    #fc_re[master] = fc_re[master].fillna(0)
    #fc_re['weekend'+master] = fc_re['ds'+master].apply(weekend)
    #fc_re = fc_re[fc_re['weekend'+master] != 1]

    test_new = round(test_new)
    test_new_df = test_new[['ds', 'yhat']].copy()
    test_new_df['y']=test_new['yhat'] #copy the log-transformed data to another column
    
    master_index+=1
print('Forecasting done')
#print('fc_re ', fc_re)
print('-------------grouped_df_transposed_re--------------------')
print(grouped_df_transposed_re)

#to add later on
#fc_re['weekend'] = fc_re['ds'].apply(weekend)
#fc_re
#fc_re = fc_re[fc_re.weekend != 1]
#fc_re
#fc_re

from datetime import timedelta, date
import pandas as pd
#date(2019, 12, 31)
start_date = date(2017, 1, 3)
end_date = today_date
histdates = pd.DataFrame()
histdates = pd.concat([pd.DataFrame({'Date': pd.date_range(start_date, end_date, freq='d')}, columns=['Date'])])
histdates

histdates = histdates[:-1]
histdates

# removing weekend from future dates - test_df_all
test_df_all['weekend'] = test_df_all['Date'].apply(weekend)
test_df_all = test_df_all[test_df_all.weekend != 1]
test_df_all
test_df_all = test_df_all.drop(['weekend'],axis=1)
test_df_all
test_df_all.index = range(0,len(test_df_all ))
test_df_all

# Merge forecasts with given IDs for Model 1
merge_all = pd.merge(histdates, test_df_all, on=['Date'] , how='outer')
#cols = ['ds','yhat']
#test_new = test_merge[cols]
merge_all['Date']

import numpy

dates = pd.to_datetime(pd.Series(merge_all['Date']), format = '%Y%m%d')
dates = dates.apply(lambda x: x.strftime('%Y-%m-%d'))
#print(len(group_code_unique))
new_re = pd.DataFrame()
new_re['Date'] = merge_all['Date']
for k in range(len(group_code_unique)):
    fc_redates = pd.to_datetime(pd.Series(fc_re['ds'+group_code_unique[k]][fc_re['ds'+group_code_unique[k]].notnull()]))
    fc_redates = fc_redates.apply(lambda x: x.strftime('%Y-%m-%d'))
    i=0
    new_df_fc_redates = pd.DataFrame(columns =['Date',group_code_unique[k]])
    for single in dates:
        vals = fc_redates[fc_redates.astype(str).str.contains(single)]
        if (len(vals) == 1):
            vals =  pd.to_datetime(vals.values)
            value = fc_re.loc[i,group_code_unique[k]]
            new_df_fc_redates = new_df_fc_redates.append({'Date':vals.values, group_code_unique[k]:value}, ignore_index=True)
        else:
            i=i-1
            vals =  pd.to_datetime(vals.values)
            new_df_fc_redates = new_df_fc_redates.append({'Date':vals.values, group_code_unique[k]:0}, ignore_index=True)
        i+=1
    new_re[group_code_unique[k]] = new_df_fc_redates[group_code_unique[k]]
new_df_fc_redates

new_re['ds'] = new_re['Date']
new_re

#grouped_df_transposed_re 
new_re_grouped = pd.DataFrame()
grouped_df_transposed_re = grouped_df_transposed_re.reset_index()
print(grouped_df_transposed_re)
new_re_grouped['Date'] = histdates['Date']
for k in range(len(group_code_unique)):
    print(pd.Series(grouped_df_transposed_re['ds'+group_code_unique[k]]))
    
    grouped_redates = pd.to_datetime(pd.Series(grouped_df_transposed_re['ds'+group_code_unique[k]][grouped_df_transposed_re['ds'+group_code_unique[k]].notnull()]))
    grouped_redates = grouped_redates.apply(lambda x: x.strftime('%Y-%m-%d'))
    i=0
    new_df_grouped_redates = pd.DataFrame(columns =['Date',group_code_unique[k]])
    for single in dates:
        vals = grouped_redates[grouped_redates.astype(str).str.contains(single)]
        if (len(vals) == 1):
            vals =  pd.to_datetime(vals.values)
            value = grouped_df_transposed_re.loc[i,group_code_unique[k]]
            new_df_grouped_redates = new_df_grouped_redates.append({'Date':vals.values, group_code_unique[k]:value}, ignore_index=True)
        else:
            i=i-1
            vals =  pd.to_datetime(vals.values)
            new_df_grouped_redates = new_df_grouped_redates.append({'Date':vals.values, group_code_unique[k]:0}, ignore_index=True)
        i+=1
    new_re_grouped[group_code_unique[k]] = new_df_grouped_redates[group_code_unique[k]]
new_df_grouped_redates

grouped_df_transposed_re = grouped_df_transposed_re.drop(['index'],axis=1)
grouped_df_transposed_re

new_re_grouped = new_re_grouped.set_index('Date')
new_re_grouped = new_re_grouped.fillna(0)

output = new_re.transpose()
output.columns = new_re['ds']
new_header = output.iloc[0] #grab the first row for the header
reoutput = output[1:] #take the data less the header row
reoutput.columns = new_header #set the header row as the df header
reoutput = reoutput.drop(['ds'],axis = 0)
reoutput = reoutput.astype('float64')
grouped_df_transposed.columns = pd.to_datetime(grouped_df_transposed.columns)
new_values =reoutput.copy()
reoutput.update(new_re_grouped.transpose())
vals = new_re_grouped.iloc[:,0:]
vals.to_dict('series')
dataframe = pd.DataFrame(new_values)
diff = dataframe.subtract(vals.transpose(), axis = 1)
diff =abs(diff)
# Absolute Values - sum over the column axis. 
sum_absolute_vals = diff.sum(axis = 1, skipna = True)
sum_absolute_vals
diff
new_values_trans = new_values.transpose()
new_values_trans

#Actual Values
sum_actual_vals = reoutput.sum(axis = 1, skipna = True)
sum_actual_vals

#MAPE Calculation
mape_error_rate = sum_absolute_vals.div(sum_actual_vals)
mape_error_rate


reoutput.insert(0, "Error Rate", mape_error_rate, True) 
writer = ExcelWriter('forecast_customer_orders_daily.xlsx')
reoutput.to_excel(writer,'Sheet1',index=True)
writer.save()


reoutput_month = pd.DataFrame()
reoutput_month = reoutput.transpose()
reoutput_month = reoutput_month.set_index(pd.to_datetime(reoutput.transpose().index))
reoutput_month = reoutput_month.resample('1M').sum()
reoutput_month

import calendar
reoutput_month['month'] = pd.to_datetime(reoutput_month.index).month
reoutput_month['year'] = pd.to_datetime(reoutput_month.index).year
reoutput_month['month'] 
reoutput_month['month'] = reoutput_month['month'].apply(lambda x: calendar.month_abbr[x])
reoutput_month['month'] = reoutput_month['month'].astype(str)
reoutput_month['year'] = reoutput_month['year'].astype(str)
reoutput_month['Period'] = reoutput_month['month']+ ', ' + reoutput_month['year']
reoutput_month['Period']
reoutput_month = reoutput_month.reset_index()
reoutput_month = reoutput_month.drop(['month','year','Date'], axis=1)
reoutput_month = reoutput_month.set_index('Period')
reoutput_month

writer = ExcelWriter('forecast_customer_orders_monthly.xlsx')
reoutput_month.transpose().to_excel(writer,'Sheet1',index=True)
writer.save()
